In [1]:


import jieba
import pandas as pd
import numpy as np
import gensim
from gensim import corpora
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [4]:
# 读取文本文件
with open('/data1/dxw_data/llm/mkt_llm/starbuck/starbuck_comments_1.txt', 'r', encoding='utf-8') as file:
    long_text = file.read()

# 停用词表
stop_words = set()
with open('/data1/dxw_data/llm/ML/LIWC/datasets/stopwords_cn.txt', 'r', encoding='utf-8') as file:
    for line in file:
        stop_words.add(line.strip())

# 文本预处理
def preprocess(text):
    # 使用jieba分词
    tokens = jieba.lcut(text)
    #! 只保留两个字的词语
    tokens = [word for word in tokens if word not in stop_words and len(word) == 2 and word.isalpha()]
    return tokens

In [5]:
# 对文本进行预处理
processed_text = preprocess(long_text)

# 创建词典和语料库
dictionary = corpora.Dictionary([processed_text])
corpus = [dictionary.doc2bow(processed_text)]

# 构建LDA模型
num_topics = 5  # 设定主题数
lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.608 seconds.
Prefix dict has been built successfully.


In [6]:
# 输出主题
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic: {idx}\nWords: {topic}\n")

# 修正后的LDA可视化部分
def prepare_lda_vis_data(model, corpus, dictionary):
    # 提取LDA模型数据
    topic_term_dists = np.array(model.get_topics())
    doc_topic_dists = np.array([[topic_prob for _, topic_prob in model.get_document_topics(bow, minimum_probability=0)] for bow in corpus])
    doc_lengths = [sum(count for _, count in doc) for doc in corpus]
    vocab = list(dictionary.token2id.keys())
    term_frequency = np.array(list(dictionary.dfs.values()))
    
    return pyLDAvis.prepare(
        topic_term_dists=topic_term_dists,
        doc_topic_dists=doc_topic_dists,
        doc_lengths=doc_lengths,
        vocab=vocab,
        term_frequency=term_frequency,
        sort_topics=False
    )



Topic: 0
Words: 0.004*"好吃" + 0.004*"老婆" + 0.004*"不错" + 0.004*"面包" + 0.004*"喜欢" + 0.004*"真的" + 0.004*"推荐" + 0.004*"口味" + 0.004*"味道" + 0.004*"早餐"

Topic: 1
Words: 0.005*"好吃" + 0.005*"老婆" + 0.004*"不错" + 0.004*"面包" + 0.004*"真的" + 0.004*"早餐" + 0.004*"喜欢" + 0.004*"味道" + 0.004*"蛋糕" + 0.004*"感觉"

Topic: 2
Words: 0.005*"好吃" + 0.005*"老婆" + 0.004*"面包" + 0.004*"不错" + 0.004*"喜欢" + 0.004*"推荐" + 0.004*"真的" + 0.004*"味道" + 0.004*"一口" + 0.004*"每次"

Topic: 3
Words: 0.047*"好吃" + 0.040*"老婆" + 0.028*"面包" + 0.026*"不错" + 0.021*"喜欢" + 0.017*"推荐" + 0.017*"真的" + 0.015*"味道" + 0.013*"口味" + 0.013*"蛋挞"

Topic: 4
Words: 0.005*"老婆" + 0.005*"好吃" + 0.004*"不错" + 0.004*"喜欢" + 0.004*"面包" + 0.004*"推荐" + 0.004*"蛋挞" + 0.004*"味道" + 0.004*"真的" + 0.004*"口味"

